<a href="https://colab.research.google.com/github/guivieiratorres/portifolio/blob/master/gerenciador_de_download_para_esta%C3%A7%C3%B5es_RBMC_IBGE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install geopandas
import math

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.0 MB 7.8 MB/s 
     |████████████████████████████████| 16.7 MB 10.5 MB/s 
     |████████████████████████████████| 6.3 MB 32.2 MB/s 


In [2]:
import pandas as pd

df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/dados/estacoes_rbmc.csv", encoding = 'utf-8', delimiter = ';')

df.head(150)


,id,Nome_est,local,lat_aprox,lng_aprox
0,1,ALAR,ARAPIRACA,-9.749225,-36.653420
1,2,ALMA,MACEIÓ,-9.554711,-35.777101
2,3,AMCO,COARI,-4.871989,-65.333979
3,4,AMCR,COARI - IBGE,-4.085823,-63.142204
4,5,AMHA,HUMAITÁ - UEA,-7.516750,-63.028401
...,...,...,...,...,...
145,146,TOPL,PALMAS,-10.171054,-48.330679
146,147,UBA1,UBATUBA(MARÉGRAFO),-23.500177,-45.118901
147,148,UBE1,UBERLÂNDIA - REDE CEMIG,-18.889536,-48.316972
148,149,UFPR,CURITIBA-UFPR,-25.448369,-49.230955


In [3]:
# CALCULO PARA CONVERTER COORDENADAS GEODESICAS EM CARTESIANAS TRIDIMENSIONAIS

from numpy import radians


def geo_to_cart3d(lat,lng,h):

#FORMULAÇÃO GEODESICA (PARÂMETROS DO ELIPSOIDE GRS80)

  a = 6378137 #semi eixo maior
  b = 6356752.3141 #semi eixo menor
  e1 = 0.00669438002290 #1º excentricidade
  e2 = 0.00673949677548 #2º excentricidade
  f = 0.00335281068118 #achatamento

  PN = a*(1 - e1)/(1 - e1 * (math.sin(radians(lat)))**2)**0.5 # Pequena Normal
  GN = a/(1 - e1 * (math.sin(radians(lat)))**2)**0.5 # Grande Normal
  X = (GN + h) * math.cos(radians(lat)) * math.cos(radians(lng))
  Y = (GN + h) * math.cos(radians(lat)) * math.sin(radians(lng))
  Z = (PN + h) * math.sin(radians(lat))
  return (X,Y,Z)

In [4]:
from numpy.ma.core import sqrt
# FUNCAO PARA CALCULAR A DISTANCIA ENTRE COORDENADAS CARTESIANAS 3D
def calc_dist(x2,x1,y2,y1,z2,z1):
  return math.sqrt((x2 - x1)**2 + (y2 - y1)**2 + (z2 - z1)**2)
  

In [5]:
# SCRIPT PARA IDENTIFICAR O DIA DO RASTREIO
from datetime import datetime

data = datetime.strptime(str(input("Data do rastreio GNSS: (dd/mm/aaaa)")), "%d/%m/%Y") #data do rastreio que deseja-se fazer download
data_zero = datetime.strptime("01/01/" + str(data)[:4], "%d/%m/%Y") #data zero para contagem

calc_data_preliminar = data - data_zero #calculo preliminar para definir os dias corridos da data
calc_data = int(str(calc_data_preliminar)[:3]) + 1 #calculo para definir os dias corridos da data
print(calc_data)

dias = str(calc_data)+'1'

Data do rastreio GNSS: (dd/mm/aaaa)15/04/2022
105


In [6]:
lat = float(input('Digite as coordenadas geodésicas : Latitude(°)'))
lng = float(input('Digite as coordenadas geodésicas : Longitude(°)'))
h = float(input('Digite as coordenadas geodésicas : Altitude elipsoidal(m)'))

Digite as coordenadas geodésicas : Latitude(°)-48.0264656
Digite as coordenadas geodésicas : Longitude(°)-43.3158595
Digite as coordenadas geodésicas : Altitude elipsoidal(m)156


In [7]:
# Criando/plotando linhas com Nome estacao , coord cartesianas 3d , dist do ponto de analise 
'''
i=0
print('Est   |                            Coord XYZ                            |  Distância') #cabeçalho
while i < len(df.index):
  print(df.iloc[i]['Nome_est'],geo_to_cart3d(df.iloc[i]['lat_aprox'],df.iloc[i]['lng_aprox'],0),
        round(calc_dist(geo_to_cart3d(lat,lng,h)[0],geo_to_cart3d(df.iloc[i]['lat_aprox'],df.iloc[i]['lng_aprox'],0)[0],
          geo_to_cart3d(lat,lng,h)[1],geo_to_cart3d(df.iloc[i]['lat_aprox'],df.iloc[i]['lng_aprox'],0)[1],
          geo_to_cart3d(lat,lng,h)[2],geo_to_cart3d(df.iloc[i]['lat_aprox'],df.iloc[i]['lng_aprox'],0)[2]),3), sep = '  |  ' )
  i = i+1
'''

"\ni=0\nprint('Est   |                            Coord XYZ                            |  Distância') #cabeçalho\nwhile i < len(df.index):\n  print(df.iloc[i]['Nome_est'],geo_to_cart3d(df.iloc[i]['lat_aprox'],df.iloc[i]['lng_aprox'],0),\n        round(calc_dist(geo_to_cart3d(lat,lng,h)[0],geo_to_cart3d(df.iloc[i]['lat_aprox'],df.iloc[i]['lng_aprox'],0)[0],\n          geo_to_cart3d(lat,lng,h)[1],geo_to_cart3d(df.iloc[i]['lat_aprox'],df.iloc[i]['lng_aprox'],0)[1],\n          geo_to_cart3d(lat,lng,h)[2],geo_to_cart3d(df.iloc[i]['lat_aprox'],df.iloc[i]['lng_aprox'],0)[2]),3), sep = '  |  ' )\n  i = i+1\n"

In [8]:
# cria dict com key= nome das estações e value = dist. entre coordenada de entrada

i=0
st = {}
while i < len(df.index):

  st[df.iloc[i]['Nome_est']] = round(calc_dist(geo_to_cart3d(lat,lng,h)[0],geo_to_cart3d(df.iloc[i]['lat_aprox'],df.iloc[i]['lng_aprox'],0)[0],
          geo_to_cart3d(lat,lng,h)[1],geo_to_cart3d(df.iloc[i]['lat_aprox'],df.iloc[i]['lng_aprox'],0)[1],
          geo_to_cart3d(lat,lng,h)[2],geo_to_cart3d(df.iloc[i]['lat_aprox'],df.iloc[i]['lng_aprox'],0)[2]),3)
  i = i+1


# cria novo dicionario com as 5 estações mais proximas do ponto de análise. a distancia está em Km
st_ordem = {}
for i in sorted(st, key = st.get):
    st_ordem[i] = round(st[i]/1000,3)
    if len(st_ordem) == 5:
      break


In [12]:
print('5 ESTAÇÕES MAIS PRÓXIMAS PARA O RASTREIO DO DIA ' + str(data) + '\n')
for i in list(st_ordem.keys()):
  print(i, ': Dist.', st_ordem[i], 'Km')
  print('Link para download dos arquivos RINEX:')
  print("https://geoftp.ibge.gov.br/informacoes_sobre_posicionamento_geodesico/rbmc/dados/"+str(data)[:4]+"/"+str(calc_data)+"/"+i.lower()+dias+".zip") # linha para baixar os arquivos Rinex.
  print('Link para download do Relatório de Informação de Estação:')
  print("https://geoftp.ibge.gov.br/informacoes_sobre_posicionamento_geodesico/rbmc/relatorio/Descritivo_"+i+'\n')


5 ESTAÇÕES MAIS PRÓXIMAS PARA O RASTREIO DO DIA 2022-04-15 00:00:00

RSPE : Dist. 1951.277 Km
Link para download dos arquivos RINEX:
https://geoftp.ibge.gov.br/informacoes_sobre_posicionamento_geodesico/rbmc/dados/2022/105/rspe1051.zip
Link para download do Relatório de Informação de Estação:
https://geoftp.ibge.gov.br/informacoes_sobre_posicionamento_geodesico/rbmc/relatorio/Descritivo_RSPE

POAL : Dist. 2092.185 Km
Link para download dos arquivos RINEX:
https://geoftp.ibge.gov.br/informacoes_sobre_posicionamento_geodesico/rbmc/dados/2022/105/poal1051.zip
Link para download do Relatório de Informação de Estação:
https://geoftp.ibge.gov.br/informacoes_sobre_posicionamento_geodesico/rbmc/relatorio/Descritivo_POAL

SMAR : Dist. 2207.791 Km
Link para download dos arquivos RINEX:
https://geoftp.ibge.gov.br/informacoes_sobre_posicionamento_geodesico/rbmc/dados/2022/105/smar1051.zip
Link para download do Relatório de Informação de Estação:
https://geoftp.ibge.gov.br/informacoes_sobre_posicio